In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv("Combined_News_DJIA.csv")

In [3]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

In [4]:
len(test)

378

In [7]:
len(train)

1611

In [8]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [9]:
trainvect = CountVectorizer()

In [10]:
Trainfeature = trainvect.fit_transform(trainheadlines)

In [9]:
####Detailed view of Document Count Matrix
DTM_With_Colm = pd.DataFrame(Trainfeature.toarray(),columns= trainvect.get_feature_names())

In [11]:
Trainfeature.shape

(1611, 31675)

Model Logistic Regression

In [12]:
Logis = LogisticRegression()

In [13]:
Model1 = Logis.fit(Trainfeature,train['Label'])

C:\Users\subhashree hazra\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
Model1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
testheadlines =[]
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [16]:
len(testheadlines)

378

In [17]:
Testfeature = trainvect.transform(testheadlines)

In [18]:
Testfeature.shape

(378, 31675)

In [19]:
Predicted = Model1.predict(Testfeature)

In [20]:
Predicted.shape

(378,)

In [21]:
pd.crosstab(test["Label"], Predicted , rownames=["Actual"] , colnames= ["Predict"])

Predict,0,1
Actual,,
0,61,125
1,92,100


Model Naive Bayes

In [22]:
from sklearn.naive_bayes import MultinomialNB

Nb = MultinomialNB()

In [23]:
Model2 = Nb.fit(Trainfeature,train['Label'])

In [24]:
Nbpredicted = Model2.predict(Testfeature)

In [25]:
Nbpredicted.shape

(378,)

In [26]:
pd.crosstab(test["Label"], Nbpredicted, rownames= ["Acutal"],colnames=["Predict"])

Predict,0,1
Acutal,,
0,46,140
1,51,141


In [27]:

import numpy as np
from sklearn.metrics import accuracy_score
y_NaviBayes = Nbpredicted
y_true = test["Label"]
accuracy_score(y_NaviBayes,y_true)
x_Logist = Predicted
x_true = test["Label"]
accuracy_score(x_Logist,x_true)


0.42592592592592593

**Ngram Model**

In [28]:
advvect = CountVectorizer(ngram_range=(1,2))
%time
advancedtrain = advvect.fit_transform(trainheadlines)
advancedtrain.shape


Wall time: 0 ns


(1611, 398396)

Model Naive Bayes - Ngram

In [29]:
advmodel = MultinomialNB()
advancemodel = advmodel.fit(advancedtrain,train["Label"])
advancetest = advvect.transform(testheadlines)
advNBprediction = advmodel.predict(advancetest) 
advNBprediction.shape




(378,)

In [30]:
pd.crosstab(test["Label"],advNBprediction, rownames=["Acutal"],colnames=["Predicted"])

Predicted,0,1
Acutal,,
0,3,183
1,0,192


In [31]:
x_adNB = advNBprediction
x_test = test["Label"]

In [32]:
accuracy_score(x_test,x_adNB)

0.5158730158730159

In [33]:
from sklearn import metrics
metrics.accuracy_score(x_test,x_adNB)

0.5158730158730159

**Latent Dirichlet Allocation**
Topic Modeling 


In [52]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [53]:
%time
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
#Our Document
trainheadlines

# list for tokenized documents in loop
texts = []

# loop through document list
for i in trainheadlines:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

Wall time: 101 ms


In [54]:
%time
#generate LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=1,chunksize=10000,update_every=1)

Wall time: 0 ns


In [55]:
import pyLDAvis.gensim
%time
print(ldamodel.print_topics(num_topics=10, num_words=3))

Wall time: 0 ns
[(0, '0.017*"s" + 0.007*"b" + 0.007*"say"'), (1, '0.035*"b" + 0.016*"s" + 0.005*"say"'), (2, '0.021*"b" + 0.017*"s" + 0.006*"us"'), (3, '0.014*"s" + 0.011*"b" + 0.005*"us"'), (4, '0.019*"s" + 0.007*"b" + 0.006*"year"'), (5, '0.033*"b" + 0.012*"s" + 0.005*"us"'), (6, '0.017*"s" + 0.005*"b" + 0.005*"say"'), (7, '0.033*"b" + 0.014*"s" + 0.006*"us"'), (8, '0.033*"b" + 0.015*"s" + 0.006*"say"'), (9, '0.021*"s" + 0.018*"b" + 0.005*"us"')]


In [56]:
ldamodel.print_topics(5)

[(9,
  '0.021*"s" + 0.018*"b" + 0.005*"us" + 0.005*"say" + 0.005*"year" + 0.004*"kill" + 0.004*"israel" + 0.004*"govern" + 0.004*"new" + 0.004*"u"'),
 (1,
  '0.035*"b" + 0.016*"s" + 0.005*"say" + 0.005*"govern" + 0.005*"will" + 0.004*"us" + 0.004*"year" + 0.004*"new" + 0.004*"world" + 0.004*"isra"'),
 (3,
  '0.014*"s" + 0.011*"b" + 0.005*"us" + 0.005*"say" + 0.005*"year" + 0.005*"kill" + 0.004*"china" + 0.004*"world" + 0.004*"new" + 0.004*"protest"'),
 (8,
  '0.033*"b" + 0.015*"s" + 0.006*"say" + 0.005*"kill" + 0.005*"israel" + 0.005*"year" + 0.004*"protest" + 0.004*"countri" + 0.004*"us" + 0.004*"new"'),
 (4,
  '0.019*"s" + 0.007*"b" + 0.006*"year" + 0.005*"say" + 0.004*"polic" + 0.004*"us" + 0.004*"will" + 0.004*"russia" + 0.003*"china" + 0.003*"protest"')]

In [58]:

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
news = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary)



C:\Users\subhashree hazra\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [59]:
news

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.000339 -0.002627       1        1  25.266338
8     -0.008660 -0.000788       2        1  21.628956
3      0.004843  0.001239       3        1   9.769872
7     -0.008151 -0.001200       4        1   9.019284
1     -0.008356 -0.000723       5        1   8.862262
4      0.008439 -0.006214       6        1   7.922798
5     -0.007441  0.003422       7        1   6.849905
2     -0.001454  0.002737       8        1   5.270893
0      0.008740 -0.000252       9        1   3.822563
6      0.012378  0.004407      10        1   1.587130, topic_info=     Category          Freq     Term         Total  loglift  logprob
18    Default  11792.000000        b  11792.000000  30.0000  30.0000
136   Default   8564.000000        s   8564.000000  29.0000  29.0000
139   Default   2654.000000      say   2654.000000  28.0000  28.0000
180   Default   1747.000000     will   1747.000000  27.0000  27.0000
184   Default   2374.000000     year   2374.000000  26.0000  26.0000
183   Default   1823.000000    world   1823.000000  25.0000  25.0000
171   Default   2421.000000       us   2421.000000  24.0000  24.0000
674   Default   1822.000000   govern   1822.000000  23.0000  23.0000
34    Default   1732.000000    china   1732.000000  22.0000  22.0000
261   Default   1569.000000    polic   1569.000000  21.0000  21.0000
89    Default   2150.000000     kill   2150.000000  20.0000  20.0000
104   Default   1840.000000      new   1840.000000  19.0000  19.0000
277   Default   1251.000000    state   1251.000000  18.0000  18.0000
384   Default   1079.000000    right   1079.000000  17.0000  17.0000
117   Default   1557.000000    peopl   1557.000000  16.0000  16.0000
17    Default   1293.000000   attack   1293.000000  15.0000  15.0000
124   Default   1692.000000  protest   1692.000000  14.0000  14.0000
79    Default   1136.000000     iran   1136.000000  13.0000  13.0000
12    Default    773.000000      amp    773.000000  12.0000  12.0000
0     Default   1253.000000      000   1253.000000  11.0000  11.0000
83    Default   2020.000000   israel   2020.000000  10.0000  10.0000
134   Default   1369.000000   russia   1369.000000   9.0000   9.0000
1490  Default    811.000000    court    811.000000   8.0000   8.0000
176   Default   1417.000000      war   1417.000000   7.0000   7.0000
1211  Default    974.000000    korea    974.000000   6.0000   6.0000
58    Default   1091.000000     forc   1091.000000   5.0000   5.0000
40    Default   1505.000000  countri   1505.000000   4.0000   4.0000
632   Default   1076.000000       uk   1076.000000   3.0000   3.0000
1425  Default   1056.000000     call   1056.000000   2.0000   2.0000
776   Default   1182.000000      one   1182.000000   1.0000   1.0000
...       ...           ...      ...           ...      ...      ...
180   Topic10     28.918837     will   1747.238037   0.0419  -5.6338
361   Topic10     15.872807      man    813.973145   0.2059  -6.2337
110   Topic10     14.010556      old    695.972961   0.2377  -6.3585
428   Topic10     12.741786     anti    615.503906   0.2657  -6.4534
261   Topic10     26.123043    polic   1569.208252   0.0477  -5.7355
3     Topic10     11.995550        2    572.532166   0.2777  -6.5138
0     Topic10     21.903721      000   1253.887207   0.0959  -5.9116
220   Topic10     17.343468    first    924.442505   0.1673  -6.1451
139   Topic10     38.146404      say   2654.830322  -0.0995  -5.3569
374   Topic10     13.457378    power    671.776978   0.2328  -6.3988
107   Topic10     15.037407  nuclear    783.427795   0.1901  -6.2878
183   Topic10     27.711039    world   1823.911865  -0.0437  -5.6765
58    Topic10     18.991415     forc   1091.254150   0.0921  -6.0543
34    Topic10     25.999447    china   1732.891724  -0.0562  -5.7402
401   Topic10     15.757508     time    847.670898   0.1581  -6.2410
117   Topic10     23.895033    peopl   1557.848389  -0.0341  -5.82